In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer

from sklearn.base import clone
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
df = pd.read_excel("ESCAPE-NA1.xlsx")
df.head()

In [ ]:
imparied = (df["moca"] < 23).sum()

not_imparied = (df["moca"] >= 23).sum()


print ("MoCA < 23 where they are imparied :", imparied)
print ("Moca >= 23 where they are not imparied:", not_imparied)

In [ ]:
df.info()

In [ ]:
# Drop the coloumns that we do not want 


df = df.drop("test_dice_score", axis = 1) 
df = df.drop("train_dice_score", axis = 1) 
df = df.drop("ecrcl_BL", axis = 1) 
df = df.drop("ena1_id", axis = 1) 
df = df.drop("hx_smok.1", axis = 1) 
df = df.drop("hx_diab.1", axis = 1) 
df = df.drop("site_enrolment", axis = 1) 
df = df.drop("hx_anticoag", axis = 1) 


In [ ]:
df.info()

In [ ]:
#Clean the coloumn names 

df.columns = (df.columns.str.strip().str.lower().str.replace(r'[^a-z0-9]+', '_', regex=True).str.replace(r'_+$', '', regex=True)
)

In [ ]:
# Fix the region coloumn cause the naming of Canada is not correct 

if 'region' in df.columns: 
    
    df['region'] = df['region'].replace({'Canadaa' : 'Canada'})

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]
df.columns

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Convert the MoCA scores to Numeric and drop the rows without a MoCA score

df["moca"] = pd.to_numeric(df["moca"], errors="coerce")

df = df.dropna(subset=["moca"])

# Create the binary target where 1 = cogntive decline and 0 = no decline 
df["target"] = (df["moca"] < 23).astype(int)



In [ ]:
df.info()

In [ ]:
df.columns.tolist()

In [ ]:
feature_columns = [
    "region",
    "country",
    "sex",
    "race",
    "ethnic",
    "rx_weight",
    "age_calc",
    "hx_diab",
    "hx_smok",
    "hgb_bl",
    "plt_bl",
    "hct_bl",
    "ptt_bl",
    "inr_bl",
    "glc_bl",
    "na_bl",
    "k_bl",
    "cl_bl",
    "hco3_bl",
    "creat_bl",
    "egfr_bl",
    "hx_cadihd",
    "hx_chf",
    "hx_recestrk",
    "hx_paststrk",
    "hx_ich",
    "hx_cnstraum",
    "hx_majsurg",
    "hx_pvd",
    "hx_crf",
    "hx_highchol",
    "hx_afib",
    "hx_htn",

]

X = df[feature_columns]

Y = df["target"]

In [ ]:
numeric_columns = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_columns = [c for c in feature_columns if c not in numeric_columns]

print("Numeric features:", numeric_columns)
print("Categorical features", categorical_columns)

In [ ]:
# Splitting the data into train and test 

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size = 0.2,
    random_state = 42,
    stratify = Y
)

In [ ]:

# Numeric columns, replaces the missing values with the median 
transformer_numeric = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median")),("scaler", StandardScaler())]
)

# Categorical columns, replaces the missing values with the most common value
transformer_categorical = Pipeline(steps = [("imputer", SimpleImputer(strategy ="most_frequent")), ("onehot", OneHotEncoder( drop="first", handle_unknown = "ignore"))]
)


preprocess = ColumnTransformer(
    transformers = [
        ("num", transformer_numeric, numeric_columns),
        ("cat", transformer_categorical, categorical_columns)
    ]
)

In [ ]:
temp_pipeline = Pipeline([("prep", preprocess)])

temp_pipeline.fit(X_train, Y_train)

categorical_pipline = temp_pipeline.named_steps["prep"].named_transformers_["cat"]
onehot = categorical_pipline.named_steps["onehot"]

features_ohe = list(onehot.get_feature_names_out(categorical_columns))

features_all = numeric_columns + features_ohe


print("Cohort Characteristics")

print(f"Total patients: {len(df)}")
print(f"Training set: {len(X_train)}")
print(f"Test set: {len(X_test)}")

print(f"Cognitive impairment rate: {Y.mean():.1%}")
print(f"Total features after encoding done: {len(features_all)}")


In [ ]:
# L1 Model 

cs = np.logspace(-3, 2, 15)

l1_model = LogisticRegressionCV(
    Cs = cs, cv = 5, penalty = "l1", solver = "saga",
    scoring = "roc_auc", max_iter = 5000, n_jobs = -1, refit = True
)

l1_pipe = Pipeline([("prep", clone(preprocess)), ("model", l1_model)])
l1_pipe.fit(X_train, Y_train)


l1_predict_y = l1_pipe.predict(X_test)
l1_proba_y = l1_pipe.predict_proba(X_test)[:, 1]

l1_acc = accuracy_score(Y_test, l1_predict_y)
l1_auc = roc_auc_score(Y_test, l1_proba_y)

print ("L1 Model")
print(f"Accuracy: {l1_acc:.3f}")
print(f"ROC AUC: {l1_auc:.3f}")




l1_coefs = l1_pipe.named_steps["model"].coef_.ravel()
epsilon = 1e-6
l1_n_selected = np.sum(np.abs(l1_coefs) > epsilon)

print(f"Number of selected features: {l1_n_selected} / {len(features_all)}")


# Sparse predictors 

l1_coef_df = pd.DataFrame({ "feature": features_all, "coef": l1_coefs, "abs_coef": np.abs(l1_coefs)})



l1_sparse_df = l1_coef_df[l1_coef_df["abs_coef"] > epsilon].sort_values("abs_coef", ascending = False )

print(f"Sparse predictors: {len(l1_sparse_df)} / {len(features_all)}")

print("\nTop 20 Sparse Predictors:")
print(l1_sparse_df[['feature', 'coef', 'abs_coef']].head(20).to_string(index=False))




In [ ]:
# Elastic net Model 


en_model = LogisticRegressionCV(
    Cs = cs, cv = 5, penalty= "elasticnet", solver= "saga", l1_ratios = [0.3, 0.5, 0.7], scoring = "roc_auc",
    max_iter= 5000, n_jobs=-1, refit = True
)

en_pipe = Pipeline([("prep", clone(preprocess)),  ("model", en_model)])
en_pipe.fit(X_train, Y_train)

en_predict_y = en_pipe.predict(X_test)
en_proba_y = en_pipe.predict_proba(X_test)[:, 1]

en_acc = accuracy_score(Y_test, en_predict_y)
en_auc = roc_auc_score(Y_test, en_proba_y)

print(f"Accuracy: {en_acc:.3f}")
print(f"ROC AUC: {en_auc:.3f}")

print(f"Best l1 ratio: {en_pipe.named_steps['model'].l1_ratio_[0]:.2f}")


en_coefs = en_pipe.named_steps["model"].coef_.ravel()
en_n_selected = np.sum(np.abs(en_coefs) > epsilon)

print(f"Number of selected features: {en_n_selected} /{len(features_all)}")


# Sparse predictors 
en_coef_df = pd.DataFrame({

    "feature": features_all, "coef": en_coefs, "abs_coef": np.abs(en_coefs)


})

en_sparse_df = en_coef_df[en_coef_df["abs_coef"] > epsilon].sort_values("abs_coef", ascending=False)

print(f"Sparse predictors: {len(en_sparse_df)}/ {len(features_all)}")


print("\nTop 20 sparse predictors:")
print(en_sparse_df[['feature', 'coef', 'abs_coef']].head(20).to_string(index=False))


print ("Elastic Net Model")

In [ ]:
# Random Forest Model 

print ("Random Forest Model")

rf_model = RandomForestClassifier(
    n_estimators= 500, max_depth = 10, min_samples_leaf= 10, min_samples_split= 20, max_features= 'sqrt'
    , class_weight= 'balanced', random_state= 42, n_jobs = -1
)

rf_pipe = Pipeline([("prep", clone(preprocess)), ("model", rf_model)])
rf_pipe.fit(X_train, Y_train)


rf_predict_y = rf_pipe.predict(X_test)
rf_proba_y = rf_pipe.predict_proba(X_test)[:, 1]


rf_acc = accuracy_score(Y_test, rf_predict_y)
rf_auc = roc_auc_score(Y_test, rf_proba_y)


print(f"Accuracy: {rf_acc:.3f}")
print(f"ROC AUC: {rf_auc:.3f}")


rf_importance = rf_pipe.named_steps["model"].feature_importances_

# Feature importance, greater than 1% 
rf_n_important = np.sum(rf_importance > 0.01)
print(f"Number of important features (>1%): {rf_n_important}/{len(features_all)}")


# Sparse predictors 
rf_importance_df = pd.DataFrame({

    "feature" : features_all,
    "importance" : rf_importance }).sort_values("importance", ascending=False)



rf_important_df = rf_importance_df [ rf_importance_df["importance"] > 0.01] 
print(f"Important features: {len(rf_important_df)}/{len(features_all)}")


print("\n Top 20 sparse predictors:")
print(rf_important_df.head(20).to_string(index=False))




In [ ]:
# Model Comaprision 

df_comparision = pd.DataFrame({

    'Model': ['L1 (Lasso', 'Elastic net', 'Random Forest'],
    'Accuracy' : [l1_acc, en_acc, rf_acc],
    'ROC AUC': [l1_auc, en_auc, rf_auc],
    'n features selected': [l1_n_selected, en_n_selected, rf_n_important]

})

print(df_comparision.to_string(index = False))

print("\n Best model by ROC AUC:", df_comparision.loc[df_comparision['ROC AUC'].idxmax(), 'Model'
])


In [ ]:
# Bootstrap analysis 

np.random.seed(42)

n_bootstraps = 50 

l1_selection = np.zeros(len(features_all))

en_selection = np.zeros(len(features_all))

rf_selection = np.zeros(len(features_all))


for b in range(n_bootstraps):
    idx = np.random.choice(len(X_train), size = len(X_train), replace= True)
    X_boot = X_train.iloc[idx]
    Y_boot = Y_train.iloc[idx]


    # L1 model 

    l1_model_boot = LogisticRegressionCV(
        Cs = cs, cv = 5, penalty = "l1", solver = "saga",
        scoring = "roc_auc", max_iter = 5000, n_jobs = -1, refit = True
    )

    l1_pipe_boot = Pipeline([("prep", clone(preprocess)), ("model", l1_model_boot)])
    l1_pipe_boot.fit(X_boot, Y_boot)

    l1_coefs_boot = l1_pipe_boot.named_steps["model"].coef_.ravel()
    l1_selection[np.where (np.abs(l1_coefs_boot) > epsilon)[0]] +=1



    # Elastic net model 
    en_model_boot = LogisticRegressionCV(
        Cs = cs, cv = 5, penalty= "elasticnet", solver= "saga", l1_ratios = [0.3, 0.5, 0.7], scoring = "roc_auc",
        max_iter= 5000, n_jobs= -1, refit = True
    )

    en_pipe_boot = Pipeline([("prep", clone(preprocess)),  ("model", en_model_boot)])
    en_pipe_boot.fit(X_boot, Y_boot)
    en_coefs_boot = en_pipe_boot.named_steps["model"].coef_.ravel()

    en_selection[np.where(np.abs(en_coefs_boot ) > epsilon)[0]] +=1



    # Random Forest 
    rf_model_boot = RandomForestClassifier(
        n_estimators= 500, max_depth = 10, min_samples_leaf= 10, min_samples_split= 20, 
        max_features= 'sqrt'
        , class_weight= 'balanced', random_state= b, n_jobs =-1
    )

    rf_pipe_boot = Pipeline([("prep", clone(preprocess)), ("model", rf_model_boot)])
    rf_pipe_boot.fit(X_boot, Y_boot)

    rf_importance_boot = rf_pipe_boot.named_steps["model"].feature_importances_

    rf_selection[np.where(rf_importance_boot > 0.01) [0]] +=1





stability_compare = pd.DataFrame({
    'feature' : features_all,
    'L1_freq' : l1_selection / n_bootstraps,
    'Elastic_Net_freq': en_selection / n_bootstraps,
    'Random_forest_freq': rf_selection /n_bootstraps


})


cons_threshold = 0.6 

stability_compare['consensus'] = (

    (stability_compare ['L1_freq'] >= cons_threshold).astype(int) + (stability_compare['Elastic_Net_freq'] >= cons_threshold).astype(int) + 
    (stability_compare['Random_forest_freq'] >= cons_threshold).astype(int)
)


stability_compare['average_freq'] = stability_compare[['L1_freq', 'Elastic_Net_freq', 'Random_forest_freq']].mean(axis=1)
stability_compare = stability_compare.sort_values(['consensus', 'average_freq'], ascending = False)




print("Stable predictors")
print(f"L1 stable features: {(stability_compare['L1_freq'] >= cons_threshold).sum()}")
print(f"Elastic Net stable features: {(stability_compare['Elastic_Net_freq']  >= cons_threshold).sum()}")
print(f"Random Forest stable features: {( stability_compare['Random_forest_freq'] >=  cons_threshold).sum()}")


print(f" \n Consensus features (that stable in two or more of the models): {(stability_compare['consensus'] >= 2).sum()}")




In [ ]:
# Show just the top 20 consensus features 

consensus_features = stability_compare[stability_compare['consensus'] >= 2]

print("Top 20 Consensus Features")

print(consensus_features[['feature', "L1_freq", "Elastic_Net_freq", "Random_forest_freq", "consensus"]].head(20).to_string(index = False))



In [ ]:
# Show all the 37 consensus features 

print(consensus_features[['feature', "L1_freq", "Elastic_Net_freq", "Random_forest_freq", "consensus"]].to_string(index = False))


In [ ]:
# Compare the features by consensus level 

universal_consensus = stability_compare[stability_compare['consensus'] == 3]
print("Universal consensus (All three models)")
print(universal_consensus[['feature', 'L1_freq', 'Elastic_Net_freq', 'Random_forest_freq']].to_string(index = False))


two_model_consensus = stability_compare[stability_compare['consensus'] == 2]
print("2 Model Consensus")
print(two_model_consensus[['feature', 'L1_freq', 'Elastic_Net_freq', 'Random_forest_freq']].to_string(index = False))



In [ ]:
# Visualizations 

top_n_features = 20

top_features = consensus_features.head(top_n_features)

data = top_features[['L1_freq', 'Elastic_Net_freq', 'Random_forest_freq']].T

plt.figure(figsize = (12, 6))
sns.heatmap(
    data,
    xticklabels= top_features['feature'].values, 
    yticklabels= ['L1', 'Elastic Net', 'Random Forest'],
    annot = True,
    fmt = '.2f',
    cmap = 'YlOrRd',
    vmin = 0,
    vmax=1
)



plt.title('Stability of Top 20 Consensus Predictors Across Models')
plt.xlabel("Feature")
plt.ylabel("Model")

plt.tight_layout(
)


In [ ]:

top_n_features = 20

top_features = consensus_features.head(top_n_features)

data = top_features[['L1_freq', 'Elastic_Net_freq', 'Random_forest_freq']].values



fig, ax = plt.subplots(figsize=(10, 12))

sns.heatmap(
    data,
    xticklabels= ['L1', 'Elastic Net', 'Random Forest'],
    yticklabels= top_features['feature'].values, 
    annot = True,
    cmap = 'YlOrRd',
    fmt = '.2f',
    vmax=1,
    vmin = 0,
    ax = ax
)



plt.title('Stability of Top 20 Consensus Predictors Across Models')
plt.xlabel("Feature")
plt.ylabel("Model")

plt.tight_layout(
)

In [ ]:

# Stability Heatmap for just the vascular features 

vascular_features = [
    "hx_afib",
    "hx_htn",
    "hx_highchol",
    "hx_cadihd",
    "hx_chf",
    "hx_pvd",
    "hx_crf",
    "hx_recestrk",
    "hx_paststrk",
]

vascular_stability = stability_compare[
    stability_compare["feature"].isin(vascular_features)].copy()


vascular_features_order = [
    "hx_afib",
    "hx_htn",
    "hx_highchol",
    "hx_cadihd",
    "hx_chf",
    "hx_pvd",
    "hx_crf",
    "hx_recestrk",
    "hx_paststrk",
]

vascular_stability["feature"] = pd.Categorical(vascular_stability["feature"], categories= vascular_features_order, 
ordered = True)


vascular_stability = vascular_stability.sort_values("feature")


fig, ax = plt.subplots(figsize=(8, 6))

heatmap_data = vascular_stability[["L1_freq", "Elastic_Net_freq", "Random_forest_freq"]].values

sns.heatmap(
    heatmap_data,
    yticklabels=vascular_stability["feature"].values,
    xticklabels=["L1", "Elastic Net", "Random Forest"],
    annot=True,
    fmt=".2f",
    cmap="YlOrRd",
    vmax=1,
    vmin=0,
    ax=ax,
)

ax.set_title("Stability of Vascular Comorbidities Across Models")
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))


# Performance by ROC AUC comparision
axes[0].bar(['L1', 'Elastic Net', 'Random Forest'],

            [l1_auc, en_auc, rf_auc],

            color=['#2ecc71', '#3498db', '#e74c3c'])


axes[0].set_ylabel('ROC AUC')
axes[0].set_title('Model Performance: ROC AUC')

axes[0].set_ylim([0.5, 1.0])
axes[0].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)



# Feature selection comparison 
axes[1].bar(['L1', 'Elastic Net', 'Random Forest'], [l1_n_selected, en_n_selected, rf_n_important], color=['#2ecc71', '#3498db', '#e74c3c'])

axes[1].set_ylabel('Number of Features')
axes[1].set_title('Feature Sparsity')



plt.tight_layout()





In [ ]:
# Tbale of features before preprocessing 

domains_before_preprocess = {
    "Identifiers / metadata": [ "ena1_id", "site_enrolment",
    ],

    "Demographics": [ "age_calc","sex", "race", "ethnic", "region",
        "country",  "rx_weight", "moca",

    ],


    "Medical history": [ "hx_diab", "hx_diab.1", "hx_smok", "hx_smok.1", "hx_cadihd", "hx_chf",
        "hx_recestrk", "hx_paststrk", "hx_ich", "hx_cnstraum", "hx_majsurg",
        "hx_pvd", "hx_crf", "hx_highchol", "hx_afib", "hx_htn",
        "hx_anticoag",
    
    ],
    
    "Baseline laboratory values": [ "hgb_BL","plt_BL", "hct_BL", "ptt_BL","inr_BL", "glc_BL","na_BL",
        "k_BL", "cl_BL", "hco3_BL","creat_BL", "egfr_BL", "ecrcl_BL",
    ],
}

rows = []
for domain, vars_list in domains_before_preprocess.items():
    rows.append({
        "Domain": domain,
        "Variables": ", ".join(vars_list)
    })




df_before_domains = pd.DataFrame(rows)

fig, ax = plt.subplots(figsize=(14, 4)) 
ax.axis('off')

before_table = ax.table(
    cellText=df_before_domains.values,
    colLabels=df_before_domains.columns,
    cellLoc='left',
    loc='center'
)


before_table.auto_set_font_size(False)
before_table.set_fontsize(10)


before_table.auto_set_column_width(col = list(range(len( df_before_domains.columns ))))

for (row, col), cell in before_table.get_celld().items():
    if row == 0:
        cell.set_text_props(weight='bold')
        cell.set_facecolor('#f0f0f0')



plt.tight_layout()
plt.show()



In [ ]:
#Table of features after preprocessing (features dropped, duplicate columns removed, etc)

domains_after_preprocess = {
    "Demographics": [
        "region","country", "sex", "race", "ethnic",
        "rx_weight", "age_calc"
    ],

    "Baseline laboratory values": [
        "hgb_bl", "plt_bl", "hct_bl","ptt_bl", "inr_bl", "glc_bl", "na_bl", "k_bl", "cl_bl", "hco3_bl",
        "creat_bl", "egfr_bl"
    ],


    "Medical history": [
        "hx_diab", "hx_smok", "hx_cadihd","hx_chf", "hx_recestrk", "hx_paststrk", "hx_ich",
        "hx_cnstraum", "hx_majsurg", "hx_pvd", "hx_crf",
        "hx_highchol","hx_afib",  "hx_htn"
    ]
}


rows = []
for domains_after_preprocess, vars_list in domains_after_preprocess.items():
    rows.append({
        "Domain": domains_after_preprocess,
        "Predictors": ", ".join(vars_list)
    })


df_domains_after = pd.DataFrame(rows)

fig, ax = plt.subplots(figsize=(14, 3))
ax.axis('off')



after_table = ax.table(
    cellText= df_domains_after.values,
    colLabels= df_domains_after.columns,

    loc = 'center',
    cellLoc = 'left',
)



after_table.auto_set_font_size(False)
after_table.set_fontsize(10)
after_table.auto_set_column_width(col= list( range(len( df_domains_after.columns))))


for (row, col), cell in after_table.get_celld().items():

    if row == 0:
        cell.set_text_props(weight='bold')

        cell.set_facecolor('#f0f0f0')
